# Estudo sobre Previsão de Vendas Utilizando Séries Temporais

## Parte 2: Pré-processamento

### Instalar as bibliotecas necessárias

In [1]:
!pip install kagglehub

### Carrregar as bibliotecas necessárias

In [12]:
# Bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

from scipy.stats import skew, kurtosis

from datetime import datetime, date

# Mapear o Google Drive no Colab e realizar download de arquivos do COlab para máquina local
from google.colab import drive, files

# Eliminar warnings
import warnings
warnings.simplefilter("ignore", category=UserWarning)


from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

### Baixar os arquivos de dados

In [3]:
# Mapeia Google Drive
drive.mount('/content/drive')

# Cria diretório no Google Colab para armazenar a Kaggle Key
! mkdir ~/.kaggle

# Copia Kaggle Key do Google Drive para o Colab
!cp /content/drive/MyDrive/Projetos/Kaggle/kaggle.json ~/.kaggle/kaggle.json

# Faz o download da base de dados diretamente do Kaggle
! kaggle competitions download -c walmart-recruiting-store-sales-forecasting
print('Download finalizado')

# Descompacta o arquivo zip com o dataset
! unzip walmart-recruiting-store-sales-forecasting.zip > unzip_log.txt 2>&1
! rm -rf walmart-recruiting-store-sales-forecasting.zip
print('walmart-recruiting-store-sales-forecasting.zip descompactado')

# Descompacta os arquivos individuais
! unzip features.csv.zip > unzip_log.txt 2>&1
! rm -rf features.csv.zip
print('Arquivo features.zip descompactado')

! unzip train.csv.zip > unzip_log.txt 2>&1
! rm -rf train.csv.zip
print('Arquivo train.zip descompactado')

! unzip test.csv.zip > unzip_log.txt 2>&1
! rm -rf test.csv.zip
print('Arquivo test.zip descompactado')

! unzip sampleSubmission.csv.zip > unzip_log.txt 2>&1
! rm -rf sampleSubmission.csv.zip
print('Arquivo sampleSubmission.zip descompactado')

Mounted at /content/drive
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 129MB/s]
Download finalizado
walmart-recruiting-store-sales-forecasting.zip descompactado
Arquivo features.zip descompactado
Arquivo train.zip descompactado
Arquivo test.zip descompactado
Arquivo sampleSubmission.zip descompactado


### Carregar os dataframes

In [37]:
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
#df_treino = pd.read_csv('/content/drive/MyDrive/Projetos/Sales forecasting/data/modified/univariate/df_treino.csv')
#df_teste = pd.read_csv('/content/drive/MyDrive/Projetos/Sales forecasting/data/modified/univariate/df_teste.csv')

### Tratamento de valores faltantes

#### Variável features

In [24]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


##### Substituir NaN por zero nas variáveis MarkDown


In [38]:
imputer = SimpleImputer(strategy='constant', fill_value=0)
features[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']] = imputer.fit_transform(features[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']] )

In [27]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     8190 non-null   float64
 5   MarkDown2     8190 non-null   float64
 6   MarkDown3     8190 non-null   float64
 7   MarkDown4     8190 non-null   float64
 8   MarkDown5     8190 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


##### Consolidar a soma das variáveis MarkDown1 a 5 em Total_MarkDown

In [39]:
features['Total_MarkDown'] = features[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].sum(axis=1)

##### Apagar as variáveis MarkDown1 a 5

In [40]:
features.drop(['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], axis=1, inplace=True)

In [31]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Store           8190 non-null   int64  
 1   Date            8190 non-null   object 
 2   Temperature     8190 non-null   float64
 3   Fuel_Price      8190 non-null   float64
 4   CPI             7605 non-null   float64
 5   Unemployment    7605 non-null   float64
 6   IsHoliday       8190 non-null   bool   
 7   Total_MarkDown  8190 non-null   float64
dtypes: bool(1), float64(5), int64(1), object(1)
memory usage: 456.0+ KB


##### Imputação dos valores faltantes de CPI e Unemplyment

Durante a fase de exploração de dados verificamos que as variáveis acima possuem valores faltantes de 03/05/2013 a 26/07/2013.

In [41]:
features['Date'] = pd.to_datetime(features['Date'])

In [33]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Store           8190 non-null   int64         
 1   Date            8190 non-null   datetime64[ns]
 2   Temperature     8190 non-null   float64       
 3   Fuel_Price      8190 non-null   float64       
 4   CPI             7605 non-null   float64       
 5   Unemployment    7605 non-null   float64       
 6   IsHoliday       8190 non-null   bool          
 7   Total_MarkDown  8190 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(5), int64(1)
memory usage: 456.0 KB


In [42]:
null_CPI = features[features['CPI'].isnull()]
for i in null_CPI.index:
  if (features['Date'][i].year == 2013 and features['Date'][i].month == 3):
    features.loc[i, 'CPI'] = 232.773
    features.loc[i, 'Unemployment'] = 7.500
  elif (features['Date'][i].year == 2013 and features['Date'][i].month == 4):
    features.loc[i, 'CPI'] = 232.531000
    features.loc[i, 'Unemployment'] = 7.500
  elif (features['Date'][i].year == 2013 and features['Date'][i].month == 5):
    features.loc[i, 'CPI'] = 232.945
    features.loc[i, 'Unemployment'] = 7.500
  elif (features['Date'][i].year == 2013 and features['Date'][i].month == 6):
    features.loc[i, 'CPI'] = 233.504
    features.loc[i, 'Unemployment'] = 7.500
  elif (features['Date'][i].year == 2013 and features['Date'][i].month == 7):
    features.loc[i, 'CPI'] = 233.596
    features.loc[i, 'Unemployment'] = 7.400

OBS.: A fonte dos dados imputados foi o U.S. Bureau of Labor Statistics.

In [43]:
features[features['Date'] == '2013-05-03']

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Total_MarkDown
169,1,2013-05-03,66.66,3.386,232.945,7.5,False,3787.03
351,2,2013-05-03,65.26,3.386,232.945,7.5,False,20103.65
533,3,2013-05-03,69.73,3.386,232.945,7.5,False,4429.08
715,4,2013-05-03,64.18,3.369,232.945,7.5,False,29468.44
897,5,2013-05-03,67.55,3.386,232.945,7.5,False,4989.51
1079,6,2013-05-03,68.17,3.386,232.945,7.5,False,19379.83
1261,7,2013-05-03,42.36,3.573,232.945,7.5,False,6750.76
1443,8,2013-05-03,61.99,3.386,232.945,7.5,False,10224.17
1625,9,2013-05-03,66.21,3.386,232.945,7.5,False,6968.36
1807,10,2013-05-03,78.41,3.671,232.945,7.5,False,21186.37


##### Encoding da variável categórica *IsHoliday*

In [44]:
encoder = OneHotEncoder()
IsHoliday_encoded = encoder.fit_transform(features[['IsHoliday']])
IsHoliday_encoded = pd.DataFrame(IsHoliday_encoded.toarray(), columns=encoder.get_feature_names_out(['IsHoliday']))
features = pd.concat([features, IsHoliday_encoded], axis=1)
features.drop('IsHoliday', axis=1, inplace=True)
features.head()

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,Total_MarkDown,IsHoliday_False,IsHoliday_True
0,1,2010-02-05,42.31,2.572,211.096358,8.106,0.0,1.0,0.0
1,1,2010-02-12,38.51,2.548,211.242170,8.106,0.0,0.0,1.0
2,1,2010-02-19,39.93,2.514,211.289143,8.106,0.0,1.0,0.0
3,1,2010-02-26,46.63,2.561,211.319643,8.106,0.0,1.0,0.0
4,1,2010-03-05,46.50,2.625,211.350143,8.106,0.0,1.0,0.0


##### Salvar o dataset em um arquivo CSV

In [45]:
features.to_csv('features_tratado.csv', index=False)
files.download('features_tratado.csv')
print('Arquivo features_tratado.csv salvo')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo features_tratado.csv salvo


---

<br>

#### Variável